In [3]:
import os

import json as js
from openai import OpenAI
from src.assistant import *

In [4]:
api_key = None
with open("../../.api-key", "r") as f:
    api_key = f.read().splitlines()[0]
    
## TODO: remove this line to get the original api-key ##
api_key+="abc"
#################### END TODO ##########################

client = OpenAI(api_key = api_key)

**Setup assistant**

In [5]:
description = """"""
instructions = """"""
name = "Scientific-Research-Idea-Generator"
tools = [{"type": ["code_interpreter", "retrieval", "function"]}]
model = "gpt-4"

In [ ]:
my_assistant = client.beta.assistants.create(
    description=description,
    instructions=instructions,
    name=name,
    tools=tools,
    model=model
)

**Append the new assistant instance to local assistant list**

In [9]:
my_assistant_id = my_assistant.get('id')
file_path = "./data/assistant_list.json"

updated_list, error = manage_assistant_file(file_path, my_assistant)
if error:
    print(error)

**Upload Files from ./data/assistant_files/\* to the OpenAI server**

In [ ]:
assistant_file_path = "./data/assistant_files/"
files = list_assistant_file_names(assistant_file_path)
print(f"Number of files: {len(files)}")

for file_name in files:
    file_path = os.path.join(assistant_file_path, file_name)
    
    if os.path.isfile(file_path):
        client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

In [ ]:
list_files = client.files.list()
list_files = js.loads(list_files)

if len(list_files["data"]) == 0:
    print("No file was uploaded")
else:
    retrieved_files = get_assitant_files(list_files)

print(f"Retrieved {len(retrieved_files)} of files")

**Bind assistant with assistant files**

In [ ]:
original_number_of_files = len(my_assistant.get('file_ids'))

for file_id in retrieved_files:
  assistant_file = client.beta.assistants.files.create(
    assistant_id=my_assistant_id,
    file_id=file_id
  )

my_assistant = client.beta.assistants.retrieve(my_assistant_id)
new_number_of_files = len(my_assistant.get('file_ids'))
print(f"{new_number_of_files-original_number_of_files} of files added.")

**Create a Thread**

In [ ]:
thread = client.beta.threads.create()

**Add a Message to this Thread**

In [1]:
content = """"""

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=content
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=my_assistant.id
)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages)

create_and_dump_messages(messages, "./data/thread_history/history_{}".format(str(my_assistant_id)+"_"+str(thread.id)))